# 变量

In [1]:
from pyautoword import *
import time
import threading
import pyautogui
import warnings
warnings.filterwarnings('ignore')

t=threading.Thread(name='admin',target=admin)
t.start()

# 变量
xxmc = '西安工业大学'
year = '2022'
other_groupby = L('学历')
# other_groupby = False

# 读取文档
read_doc('model/红色模板.docx')
data = get_3_data(xxmc,year,'派遣')

In [2]:
#颜色设置
color_1 = '#147BC5' #主配色
color_2 = '#FBB03E' #主配色
color_3 = '#24AAE1' #主配色
color_4 = '#E3798D' 
color_5 = '#00ABCB' 

import re
import os
import wx
import sqlalchemy
import math
import functools
import numpy
import pandas as pd
from IPython.display import clear_output
from decimal import Decimal
import keyboard
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH  #设置对象居中、对齐等。
from docx.enum.text import WD_TAB_ALIGNMENT,WD_TAB_LEADER,WD_LINE_SPACING  #设置制表符等
from docx.shared import Inches   #设置图像大小
from docx.shared import Pt,Cm   #设置像素、缩进等
from docx.shared import RGBColor    #设置字体颜色
from docx.shared import Length    #设置宽度
from docx.oxml.ns import qn  #设置中文版式
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.enum.text import WD_ALIGN_PARAGRAPH
import win32com.client
import pyautogui
from pyecharts.charts import Bar,Line,PictorialBar,Map,Pie,Sankey,Grid,Gauge,Radar,WordCloud,Liquid
from pyecharts import options as opts
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot
from pyecharts.commons.utils import JsCode
from pyecharts.components import Image
from pyecharts.options import ComponentTitleOpts
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


#矢量图设置,可以在阿里巴巴矢量图网站自行查找需要的图标,复制SVG代码
bd = 'path://M945.08544 366.6432l-425.97376-183.1936a20.5312 20.5312 0 0 0-8.12544-1.6896c-2.7648 0-5.52448 0.5632-8.14592 1.6896L76.87168 366.6432c-8.98048 3.84512-14.92992 13.65504-14.92992 24.59136 0 10.9568 5.94944 20.74112 14.92992 24.60672L502.8352 599.04a20.29056 20.29056 0 0 0 16.27136 0l331.1872-142.43328v145.7152c-17.75104 9.40032-30.08512 29.8496-30.08512 53.62688 0 23.7824 12.33408 44.22656 30.08512 53.62688v50.83136h43.6224v-50.82112c17.76128-9.40032 30.1056-29.8496 30.1056-53.63712s-12.34432-44.2368-30.1056-53.63712V437.84704l51.16416-22.00576c8.97536-3.8656 14.9248-13.65504 14.9248-24.60672 0-10.93632-5.94944-20.74624-14.9248-24.59136z m-434.0992 269.16352a47.96928 47.96928 0 0 1-19.03104-3.95776L226.4064 517.632v157.70112c0 129.16224 193.65888 167.936 254.208 167.936h60.69248c45.35808 0 254.22848-38.77376 254.22848-167.936v-157.696l-265.55904 114.21184a47.76448 47.76448 0 0 1-18.9952 3.95776z'
sf = 'path://M0,10 L10,10 C5.5,10 5.5,5 5,0 C4.5,5 4.5,10 0,10 z'
male = 'path://M724.8 314.2c2.8 1.6 5.5 3.2 8.2 5.1l8.2 5.6 0.1 9.8c0.4 28.5-3 52.6-10.4 71.1-6.9 17-17.1 29.7-30.7 37.3-9.3 33.1-20.2 63.8-35.6 89.6-17.8 29.7-41.2 52.5-74.4 64.8-14.8 5.4-52.7 8-89.1 7.4-35.2-0.5-70.5-4-84.5-10.5-30.1-13.9-51.3-37.2-67.4-66.3-13.8-24.8-23.6-53.7-32.1-84.2-14.1-7.4-24.7-20-31.8-37.1-7.8-18.7-11.4-43.2-10.9-72.2l0.1-9.9 8.2-5.5c2.1-1.4 4.2-2.7 6.3-3.9-9.2-114.1-5.7-156.3 36.6-204.5 82.6-67.7 272-65.3 355.7-3.9 57 53.8 61 114.1 43.5 207.3zM551.3 644.1l1.2 26.1-15.5 25.6 21.6 141.8L647 647.5l134.7-4.6c69.6 65.8 114.2 220.8 103.2 321.9H134.2c1.9-88.8 18.2-240.4 106.6-317.6l121.7 1.1 114 188 21.4-140.6-15.5-25.5 1.2-26.1c29.5-1.6 38.2-1.6 67.7 0zM652 273.7c-53.5 10.5-133.2 19.6-196.2-15.8-24.2-13.6-59.7 14.3-88.7 11.4-9 17.8-15.7 37.3-19.6 58l-3.2 17.1-17.2-1.7c-3.2-0.3-6.5 0.1-10 1.3-1.6 0.5-3.1 1.1-4.7 1.9 0.5 19 3.1 34.7 8 46.5 4.3 10.4 10.4 17.1 18.2 19.6l10.1 3.1 2.8 10.1c8.4 31.3 17.9 60.8 31.2 84.8 12.3 22.3 28.1 40 50 50.1 9.2 4.2 38.4 6.6 69 7.1 32.5 0.5 64.9-1.3 75.6-5.2 24.1-8.9 41.4-26.1 54.9-48.6 14.5-24.2 24.8-54.9 33.8-88.6l2.6-9.6 9.6-3.3c7.6-2.7 13.5-9.6 17.6-20 4.7-11.7 7.3-27.2 7.7-45.8-1.4-0.7-2.8-1.3-4.2-1.7-3.4-1.1-6.6-1.7-9.7-1.5l-16.8 1.1-3.1-16.5c-3.8-19.2-9.7-37.2-17.7-53.8z m0 0'
female = 'path://M354.304 631.808c18.944-7.168 44.544-17.408 56.32-26.624 7.168-5.632 11.264-16.384 12.8-28.16-4.608-3.072-9.216-6.144-12.288-9.728-53.248-54.784-56.32-132.608-56.832-229.376 8.704 1.536 271.872 5.12 284.16 3.584 2.048 109.056-5.632 176.128-68.608 234.496l-1.024 1.024c2.048 12.288 6.144 23.04 12.8 28.16 12.8 9.728 35.84 20.992 51.2 28.16-29.696 108.544-66.048 172.544-103.936 195.584-5.632-20.48-6.144-44.544-0.512-72.192-4.608-2.048-9.216-3.584-14.336-5.632-5.12 6.656-9.728 13.312-13.824 20.48-4.096-6.656-8.704-13.824-13.824-20.48-4.608 2.048-9.216 3.584-14.336 5.632 5.632 29.184 5.12 54.272-1.536 75.776-41.984-19.968-82.944-88.064-116.224-200.704z m393.216 16.384c-9.216-28.672-16.384-64.512-9.728-95.232 16.384-74.24 3.584-138.752-5.12-159.744-20.992-51.2 2.56-101.888-6.656-140.8-14.848-62.464-41.984-105.984-86.016-135.68-60.928-40.96-151.552-37.888-230.4-20.48-69.12 14.848-134.656 49.152-130.56 107.008 3.072 41.984-4.608 71.168-19.968 92.16-13.824 19.456-15.872 39.424-12.8 60.416 9.216 54.272 48.128 83.968 26.624 136.704-18.944 47.616-41.984 124.928-44.544 169.472-31.232 11.776-66.56 26.112-103.936 39.936-28.672 52.736-38.4 125.952-32.768 268.8H890.88c2.56-111.616 1.024-221.184-40.448-296.96-31.744-7.168-68.608-16.896-102.912-25.6z'
#cube = 'path://M42.666667 128m53.333333 0l832 0q53.333333 0 53.333333 53.333333l0 661.333334q0 53.333333-53.333333 53.333333l-832 0q-53.333333 0-53.333333-53.333333l0-661.333334q0-53.333333 53.333333-53.333333Z'
cube = 'path://M62 62h900v900h-900v-900z'
box = 'path://M962 802H 62V 222h 900v 580z      M162 702h 793V 322H 70v 380z'
boy='path://M516.769444 0c-282.75 0-512 229.25-512 512 0 282.75 229.25 512 512 512 282.75 0 512-229.25 512-512 0-282.875-229.25-512-512-512z m0 987.375c-262.5 0-475.375-212.875-475.375-475.375 0-126.125 50.125-247 139.25-336.125S390.769444 36.5 516.769444 36.5c262.625 0 475.375 212.875 475.375 475.375 0.125 262.625-212.75 475.5-475.375 475.5z M788.144444 691.625c2.5 8 4.25 16.25 5.125 24.625 1.625 10 2.375 20 2.375 30.125 0.125 9.625-0.875 19.25-2.75 28.75-1 7.25-3.625 14.25-7.625 20.375-5.5 4.625-12.125 7.875-19.125 9.375-12.75 3.5-25.625 6.25-38.625 8.25-15.5 2.5-32.625 5-51.75 7.25-19.125 2.25-37.875 4.25-57 5.875-19.125 1.625-37.5 2.75-55.375 3.5-17.875 0.75-33.5 1-47 1s-29.125-0.5-47-1.375c-17.875-0.875-35.875-2.125-54.625-3.5-18.75-1.375-37.5-3-55.75-4.875-18.375-1.875-35.5-4-50.625-6.25s-28.25-4.5-39-6.5c-6.875-0.75-13.625-3-19.5-6.625-6.875-8.75-10.625-19.625-10.75-30.75-2.125-22.5-1-45.25 3.25-67.375 1.625-14.375 9.125-27.375 20.75-35.875 11.75-8.625 24.75-15.25 38.625-19.625 14.75-4.875 30.25-9.125 46.625-12.75 14.875-3.125 29.125-8.75 42.25-16.625 7.875-4.375 15.25-9.75 21.875-16 4.875-4.25 8.75-9.375 11.5-15.25 2-5 3.125-10.5 3.125-16s-0.375-11.75-0.75-18.625c-0.25-9-4.125-17.5-10.75-23.5-6.75-5.875-13.75-11.625-20.75-17.25-4.25-3.125-7.625-7.25-10-12-2.875-5.375-5.5-10.75-8-16.25-2.625-6.5-4.875-13.125-6.75-19.875-0.75-0.875-2.375-2.5-4.375-4.875-1.875-3-3.625-6.125-5.125-9.375-2.625-5.5-5-11.125-6.75-17-2.125-6.5-3.75-13.125-4.75-19.875-0.875-4.875-0.75-10 0.375-14.875 0.25-4.625 1.5-9 3.625-13.125 0.125-16.25 1.125-32.375 3.125-48.5 2.25-14.75 5.5-29.25 9.625-43.5 4.25-15.375 11.5-29.75 21.5-42.125 8.5-10.875 18.375-20.5 29.5-28.75 10-7.5 21-13.375 32.625-17.625 11-4.125 22.25-7 33.875-8.625 11-1.5 22-2.375 33.125-2.375 9.375 0.625 18.75 2 27.875 4 6.125 1.25 12 3.125 17.5 5.875 4.375 2.125 8.375 5 12 8.375 3.125 3.125 6.375 6.5 10 9.625h14.375c4.5 0 8.875 0.625 13.125 1.75 4.625 1.375 8.875 3.625 12.75 6.625 5 4.125 9.5 8.75 13.5 13.875 11.25 13.375 19.875 28.75 25.5 45.25 5.25 15.25 9.25 30.75 12 46.625 2.5 16.75 4.125 33.625 4.75 50.5 1 2.625 1.625 5.5 2 8.25 0.375 3.25 0.75 7.125 1.25 11.75 0.375 5.75-0.125 11.625-1.25 17.25-0.5 7.375-2 14.75-4.375 21.75-1.5 4.375-3.875 8.375-6.75 12-2.125 3-5 5.375-8.375 6.875-1.75 6.875-4 13.5-6.75 19.875-2.375 5.5-4.75 11-7.625 16.25-2.5 4.75-6 8.875-10.375 12-8.375 6.5-15.5 12-20.75 16.25-6.25 6-10.375 13.875-11.5 22.5-1.25 6.375-1.625 12.875-1.25 19.375 0.5 7 2.5 13.75 5.625 19.875 3.75 7.375 9 13.875 15.5 19 9.125 7 19.375 12.375 30.25 16 11.5 4 24.25 8 37.875 11.5 13.5 3.5 27.125 7.375 39.875 11.75 12.125 4.125 23.625 9.875 34.25 17 9.5 6.375 16.75 15.875 20.625 26.625z m-240.375-22.75c1.5-2.625 1.625-5.875 0.375-8.625-1.375-3.125-3.25-6.125-5.625-8.625-2.75-2.75-5.625-5.75-9.125-9h-34.25c-3.125 2.75-5.875 5.75-8.375 9-2 2.25-3.625 4.875-4.75 7.625-1.25 2.375-1.125 5.375 0.375 7.625 3.25 6.5 6.375 12 8.75 17 2 4.375 4.875 8.25 8.375 11.375-1.25 4-2.375 10-4.375 17.625s-4 15.5-6 23.5c-2 8.125-3.125 15.75-4.375 22.875-1 5.375-1.5 10.75-1.625 16.25 0.125 3 1.25 6 3.25 8.25 2.5 3.5 5.125 6.875 8 10 2.625 3.25 5.75 6.125 9.125 8.625 4.875 4.625 12.625 4.625 17.5 0 3.75-2.375 7.125-5 10.375-8 3.125-2.875 5.75-6.125 8-9.625 2-2.5 3.25-5.5 3.625-8.625-0.375-4.75-1.125-9.5-2.375-14.125-1.625-6.75-3.125-14.125-4.75-22.5-1.625-8.25-3.625-16.375-5.625-24.25-2-7.75-3.625-14.125-5.125-18.75 3.25-2.875 6.125-6.25 8.375-10 2.75-4.375 6.25-10.25 10.25-17.625z'
girl='path://M516.769444 0c-282.75 0-512 229.25-512 512 0 282.75 229.25 512 512 512 282.75 0 512-229.25 512-512 0-282.75-229.125-512-512-512z m0 987.375c-262.5 0-475.375-212.75-475.375-475.375C41.394444 385.875 91.519444 265 180.644444 175.875S390.769444 36.625 516.769444 36.625c262.625 0 475.375 212.875 475.375 475.375 0.125 262.625-212.75 475.375-475.375 475.375z M788.019444 687c2.625 7.875 4.5 15.875 5.625 24.125 1.25 9.875 1.875 19.75 2 29.625 0 9.5-0.75 19-2.375 28.375-0.875 7.25-3.5 14.25-7.625 20.375-5.625 4.75-12.25 7.875-19.5 9.25-10.375 3.125-23.875 6.375-39.875 9.5-15.875 3.125-33.875 6.25-53.375 9.25-19.5 3-39.375 5.5-59 7.625-19.5 2.125-38.625 3.75-57 4.875-18.375 1.125-33.875 1.625-47.375 1.625-13.5 0-28.625-0.625-46.25-2-17.5-1.25-34.625-2.875-53.375-4.75-18.75-1.875-36.25-4-54.125-6.75-17.875-2.75-34.25-5.125-49-8-14.75-2.75-27.125-5.25-37-7.625-6.875-1.125-13.5-3.375-19.5-6.75-7-8.75-10.875-19.625-11.125-30.875-2.125-22.375-1-45 3.125-67.125 2.125-14 9.75-26.625 21.125-35 11.875-8.375 25.125-14.875 39-19.125 14.75-4.625 30.25-8.875 46.25-12.75 14.875-3.25 29.125-8.875 42.25-16.5 7.5-4.125 14.625-9 21.125-14.625 4.5-3.875 8.25-8.5 11.125-13.625 2.25-4.25 3.625-8.875 4.375-13.625 0.75-5.125 0.875-10.25 0.375-15.25l-30.625-4.625h-119.5c20.5-22.25 34.375-49.875 39.875-79.625-0.25-59.375 8.375-118.375 25.5-175.25 9.5-21.5 23.375-40.75 40.625-56.625 33.875-36.625 83.625-54.125 133-47 64.125-7.25 126.5 24 159.25 79.625 24.875 63.5 38.25 130.875 39.875 199.125 0.875 31.125 15.5 60.25 39.875 79.625h-119.5l-31.875 4.75c-0.375 5.625 0 11.375 1.25 16.875 1.375 6 3.75 11.75 7.125 16.875 4.5 6.125 9.875 11.5 15.875 15.875 8.5 5.75 17.75 10.25 27.5 13.375 12.125 4.5 24.5 8.375 37 11.5 13.5 3.25 26.75 7.125 39.875 11.75 12.125 4.125 23.625 9.75 34.25 16.875 9.625 6.375 16.875 15.75 20.75 26.625z m-266.5 102.375c3.75-0.5 7.25-1.875 10.375-4 4.875-2.875 9.5-6.25 14-9.875 4.375-3.5 8.5-7.375 12.375-11.375 2.5-2.375 4.375-5.5 5.125-8.875-0.375-4.5-1-9-2-13.375-1.625-6.375-3.125-13.375-5.125-21-2-7.625-3.625-15-5.625-22.25-1.25-6-2.875-12-4.75-17.75 4.625-3.25 8.75-7.125 12.375-11.5 3-3.5 4.875-8 5.125-12.75-0.125-3.75-1.5-7.375-4-10.125-2.625-3.5-5.5-6.625-8.75-9.5-3.625-3-8-5.875-12.375-8.875h-35c-4 3-8 6-11.125 8.875-2.875 2.75-5.5 5.875-7.625 9.25-2.25 3-3.5 6.75-3.625 10.5 0.625 3.625 2.375 6.875 4.75 9.5 3.875 4.625 8.125 8.875 12.75 12.75-2 5.25-3.75 10.75-5.125 16.25-2 7-4.375 14.375-6.75 22.25-2.375 7.875-4.375 15.25-6 22.25-1.375 5.125-2.125 10.375-2.375 15.625 0.875 3.375 2.625 6.5 5.125 8.875 4 4.125 8.25 7.875 12.75 11.375 4.625 3.75 9.5 7 14.75 9.875 3.25 2.125 6.875 3.5 10.75 3.875z'
location='path://M528 32C325.056 32 160 196.8 160 399.36c0 75.2 22.656 147.584 65.024 208.48 2.112 3.648 4.256 7.168 6.784 10.592l268.608 353.472c7.296 8.096 17.088 12.576 27.584 12.576 10.368 0 20.224-4.512 28.768-14.08l267.36-352c2.624-3.52 4.896-7.36 6.112-9.6A364.864 364.864 0 0 0 896 399.36C896 196.8 730.912 32 528 32z m0 498.72a131.52 131.52 0 0 1-131.456-131.232 131.488 131.488 0 0 1 262.88 0 131.52 131.52 0 0 1-131.424 131.2z'
airplane='path://M965.40416 342.76608c-19.50976-52.24704-47.744-100.41472-83.92064-143.16288-18.26688-21.58592-50.57408-24.27392-72.15872-6.0096-21.58464 18.26688-24.2752 50.5728-6.00832 72.15872C861.4336 334.42816 893.44 421.88032 893.44 512c0 210.32576-171.11424 381.44-381.44 381.44-210.32704 0-381.44-171.11424-381.44-381.44 0-210.32704 171.11296-381.44 381.44-381.44 60.48896 0 118.30784 13.72672 171.85152 40.8 25.2288 12.75648 56.03328 2.64704 68.7936-22.58816 12.76032-25.23392 2.64576-56.03456-22.58816-68.7936C662.05312 45.5936 588.68992 28.16 512 28.16c-65.29536 0-128.66432 12.8-188.34688 38.04416-57.62176 24.37248-109.36064 59.2512-153.7792 103.66976-44.41856 44.41856-79.29856 96.15744-103.66976 153.77792C40.96 383.3344 28.16 446.70464 28.16 512c0 65.29536 12.8 128.6656 38.04416 188.34688 24.37248 57.62176 59.2512 109.36064 103.66976 153.7792s96.15744 79.29856 153.7792 103.66976C383.33568 983.04 446.70464 995.84 512 995.84c65.29536 0 128.6656-12.8 188.34816-38.04416 57.62176-24.3712 109.36064-59.2512 153.77792-103.66976 44.41856-44.41856 79.29856-96.15744 103.66976-153.7792C983.04 640.6656 995.84 577.29536 995.84 512 995.84 453.79328 985.6 396.85504 965.40416 342.76608z  M718.52672 248.7872c-28.15104 3.3984-50.06336 18.60608-68.33152 39.93472-17.31456 20.21376-35.4048 39.76192-53.09312 59.65568-9.1136 10.24896-17.9328 20.76672-27.22688 30.848-3.22944 3.50464-5.80992 8.89984-12.31232 7.28448-3.84896-0.95616-7.7504-1.69984-11.59936-2.65472-36.48128-9.04064-72.96128-18.08512-109.43232-27.168-60.3712-15.03616-120.73344-30.10816-181.10592-45.14432-9.62304-2.39616-18.24896 0.9856-22.54336 9.56544-3.24864 6.49216-4.54016 14.9376 4.33152 22.69952 5.83808 5.10848 12.08832 9.75232 18.2336 14.5024 19.91424 15.39456 39.84512 30.76864 59.808 46.10048 16.80256 12.90496 33.6896 25.69984 50.48192 38.61632 19.79008 15.22432 39.51104 30.53696 59.2768 45.79328 8.96128 6.91712 17.95584 13.78944 27.40352 21.04064-19.81568 21.91744-39.24864 43.34848-58.5984 64.85376-4.78592 5.31968-8.928 11.26912-14.04416 16.22784-5.04192 4.88704-10.34752 11.35488-16.57088 12.86272-32.064 7.76576-64.45824 14.1632-96.74112 21.02912-8.75776 1.8624-13.8624 8.44928-14.24256 17.36192-0.544 12.77568 5.64864 16.6592 15.6544 20.66688 25.02016 10.0224 50.08896 19.92704 75.02592 30.15552 2.44352 1.00224 4.96896 3.5456 5.9968 5.98784 11.0208 26.20032 21.80864 52.50048 32.52992 78.82496 4.96 12.18176 18.5408 14.64064 27.68256 9.10848 7.41632-4.48896 8.10752-11.79648 9.67168-19.33568 5.04576-24.3264 10.70976-48.52608 16.34048-72.72704 2.4192-10.39616 2.5728-21.76256 12.8256-28.92416 5.67168-3.9616 10.81984-8.71168 15.99488-13.34016 21.27232-19.02336 42.45248-38.15168 63.70816-57.19424 1.73824-1.55776 3.80032-2.752 5.95072-4.288 2.752 3.44064 5.02272 6.20288 7.21152 9.02784 14.5216 18.73664 29.00352 37.50272 43.5392 56.22784 10.68288 13.76256 21.39776 27.49952 32.15232 41.20448 14.64832 18.66496 29.36576 37.27488 44.02304 55.93344 14.13248 17.9904 28.14464 36.07424 42.3808 53.97888 4.43392 5.57696 14.08384 8.33408 20.02688 5.9008 6.4384-2.6368 14.7328-8.84864 12.9088-18.06464-1.53728-7.76576-3.0528-15.55328-5.02528-23.21664-11.86048-46.09408-23.89504-92.14208-35.7824-138.22976-12.672-49.12256-25.27744-98.26176-37.76256-147.43168-0.48256-1.90208-0.07552-4.68096 1.056-6.208 2.2592-3.04768 5.19168-5.64224 8.05376-8.18944 18.61248-16.56704 37.248-33.1072 55.94496-49.57952 9.17248-8.08064 18.71104-15.76064 27.68384-24.05376 8.83712-8.16768 17.97888-16.22144 25.55648-25.49376 16.64768-20.36992 22.62272-44.4032 20.3456-70.34624-1.32224-15.0592-7.29728-28.56576-18.59968-38.81472C760.5312 248.56064 739.60704 246.24256 718.52672 248.7872zM761.84064 323.68128c1.86496 19.5456-6.144 33.90208-21.51552 44.9664-3.6032 2.59456-7.03616 2.65856-10.54464-0.34688-8.30976-7.11808-16.67712-14.17088-25.08672-21.17376-9.52448-7.93344-19.16416-15.72864-28.6592-23.69792-3.52768-2.96064-6.2976-5.32608-3.69536-11.50208 6.65472-15.78752 25.36192-30.99392 43.86688-29.17632C737.91744 281.6704 759.61984 300.41984 761.84064 323.68128z'
star = 'path://M956.741665 419.885046c-5.241374-16.200984-19.178805-28.054963-36.034704-30.496571l-254.931157-44.791136L551.77233 115.15981c-7.444553-15.248286-22.992667-24.898063-40.025599-24.898063-17.036002 0-32.522718 9.648754-40.146349 24.898063L357.655238 344.597339 102.784456 390.936737c-16.856923 2.442632-30.852682 14.295588-36.035728 30.496571-5.300726 16.141632-0.893346 33.951253 11.317767 45.804209l184.407963 170.410158-48.186466 247.662613c-2.859118 16.737196 3.930519 33.833573 17.809621 43.720757 13.244652 13.244652 40.562834 6.90834 47.053666 3.514033l232.595452-113.64634L742.85225 932.545078c6.490831 3.394307 32.166607 11.373025 47.052642-3.514033 13.760398-9.887184 20.787442-26.982538 17.869996-43.720757L761.077334 637.647675l184.467315-172.018795C957.69641 453.836299 962.043415 436.026678 956.741665 419.885046z'

In [8]:
%load_ext autoreload
%autoreload 2

# 小函数

In [9]:
%aimport pyautoword
read_doc('model/红色模板.docx')

from pyautoword import add_table,add_pic

In [79]:
def xy(datat,x,x1,y):
    return datat[datat[x]==x1][y].values[0]

def is100(group):
    s = sum(group)
    temp=0
    for i in group:
        temp+=round(i/s,4)
    return temp==1

def get_og_name(string,datat):
    return list(filter(lambda x:string in x,(list(datat.columns))))
    

# 第一章     毕业生就业基本情况

## 一、毕业生规模及结构

### （一）本科毕业生规模

In [62]:
write_zw(f'''本部分数据来源于{xxmc}{year}届毕业生派遣数据库，主要从毕业生规模与结构、初次毕业去向落实情况、单位就业流向三个方面进行分析。''')
add_head(2,"一、毕业生规模与结构")
add_head(3,"（一）毕业生规模")

In [63]:
datat = get_group_order_table(data,L('性别'),other_groupby)#获得分组的data
bys_count = xy(datat,L('性别'),'总计','num')
boy_count = xy(datat,L('性别'),'男','num')
girl_count = xy(datat,L('性别'),'女','num')

,xb,本科_num,本科_proportion,研究生_num,研究生_proportion,num,proportion
0,性别,本科毕业生,left,硕士毕业生,left,全体毕业生,left
1,up,人数,占比,人数,占比,人数,占比
2,女,1964,43.48%,376,44.34%,2340,43.62%
3,男,2553,56.52%,472,55.66%,3025,56.38%
4,总计,4517,100.00%,848,100.00%,5365,100.00%


In [64]:
if other_groupby:
    string=f'毕业生总人数{bys_count}人。其中，'
    group2=get_og_name('_num',datat)
    for x in group2:
        string+=f"{xy(datat,L('性别'),'性别',x)}{xy(datat,L('性别'),'总计',x)}人，占毕业生总人数的{change_to_decimal(xy(datat,L('性别'),'总计',x)/bys_count*100)+'%'}，"
    string = string[:-1]+'。'
    write_zw(string)
    string=f"分性别来看，男生{boy_count}人，占毕业生总人数的{xy(datat,L('性别'),'男','proportion')}；女生{girl_count}人，占毕业生总人数的{xy(datat,L('性别'),'女','proportion')}。"
    for x in group2:
        string+=f"{xy(datat,L('性别'),'性别',x)}男女比例为{round((xy(datat,L('性别'),'男',x)/xy(datat,L('性别'),'女',x))*100)}：100，"
    string+=f"全体毕业生男女比例为{round((boy_count/girl_count)*100)}：100。"
    write_zw(string)
    group=get_og_name('num',datat)
    group1=list(datat.loc[0][group])
    a=my_bar(group1,[['男',[xy(datat,L('性别'),'男',x) for x in group],color_1],['女',[xy(datat,L('性别'),'女',x) for x in group],color_2]])
    b=my_pie([*zip(list(datat.loc[0][group2]),[xy(datat,L('性别'),'总计',x) for x in group2])])
    if not is100([xy(datat,L('性别'),'总计',x) for x in group2]):pyautogui.confirm(text="百分比不为100",title="警告")
    c=merge_pic(a,b)
    display(c.render_notebook())
    add_pic(c.render())
else:
    write_zw(f'''毕业生总人数{bys_count}人。其中，男生{boy_count}人，占毕业生总人数的{xy(datat,L('性别'),'男','proportion')}；女生{girl_count}人，占毕业生总人数的{xy(datat,L('性别'),'女','proportion')}。毕业生男女比例为{round((boy_count/girl_count)*100)}：100。''')
    a=my_liquid(bys_count,[['男生',boy_count,color_1,["30%", "50%"]],['女生',girl_count,color_2,["70%", "50%"]]])
    display(a.render_notebook())
    add_pic(a.render())
write_tz('毕业生规模及性别分布')
#注：既有数字又有百分比的圆环图是没办法做到数字和百分比不统一的，我只能想到的方法是做出两张图，一张是正确的数字一张是需要的百分比，然后把需要覆盖的百分比截图与原图匹配挖空用需要的填充

### （二）毕业生结构分布

#### 1.学院结构

In [119]:
add_head(3,"（二）毕业生结构分布")
add_head(4,"1.学院结构")

string='学院'
data1 = get_group_order_table(data,L(string))
write_zw(f'''毕业生人数排名前三的{string}分别为{data1[L(string)][1]}（{data1['num'][1]}人，占比{data1['proportion'][1]}）、{data1[L(string)][1]}（{data1['num'][2]}人，占比{data1['proportion'][2]}）、{data1[L(string)][1]}（{data1['num'][3]}人，占比{data1['proportion'][3]}）。''')
if not other_groupby:
    data1 = get_group_order_table(data,L(string),other_groupby)
    string1 = f'''分{L(other_groupby)}来看，'''
    for i in get_og_name('_num',data1):
        string1+=f"{xy(data1,L(string),string,i)}人数排名前三的学院分别为"
        datat=data1.loc[2:][[L(string),i,re.sub('num','proportion',i)]]
        datat=datat[datat[i]!='-']
        datat=datat.sort_values(by=i,ascending=False)[1:4]
        for j in range(datat.shape[0]):
            string1+=f"{datat.iloc[j][0]}（{datat.iloc[j][1]}人，占比{datat.iloc[j][2]}）、"
        string1=string1[:-1]+'；'
    string1=string1[:-1]+'。'
    write_zw(string1)
add_table(data1)#data转化为表格

,yx,num,proportion
0,学院,人数,占比
1,电子信息工程学院,753,14.03%
2,经济管理学院,702,13.08%
3,计算机科学与工程学院,683,12.73%
4,机电工程学院,675,12.58%
5,光电工程学院,583,10.87%
6,材料与化工学院,501,9.34%
7,艺术与传媒学院,414,7.72%
8,文学院,323,6.02%
9,建筑工程学院,319,5.95%


#### 2.专业结构

In [135]:
add_head(4,"2.专业结构")
write_zw(f'''毕业生人数排名前三的专业分别为{df_zyjg_order.iloc[0]['zy']}（{df_zyjg_order.iloc[0]['rs']}人，占比{df_zyjg_order.iloc[0]['ratio']:.2f}%）、{df_zyjg_order.iloc[1]['zy']}（{df_zyjg_order.iloc[1]['rs']}人，占比{df_zyjg_order.iloc[1]['ratio']:.2f}%）、{df_zyjg_order.iloc[2]['zy']}（{df_zyjg_order.iloc[2]['rs']}人，占比{df_zyjg_order.iloc[2]['ratio']:.2f}%）。''')

NameError: name 'add_headd' is not defined

In [20]:
from pyecharts.commons.utils import JsCode

def my_pie1(group):
    group2=[x[1] for x in group]
    group1=sum100(group2)
    group=[*zip([x[0] for x in group],[*zip(group2,group1)])]
    b = (
         Pie()
        .add("",group,radius=["30%", "55%"],center=["25%", "55%"],
                label_line_opts=opts.PieLabelLineOpts(length_2=40,linestyle_opts=opts.LineStyleOpts(width=2.5)),
                label_opts=opts.LabelOpts(interval=0,
                    position="outside",
                    formatter=JsCode(
                    "function(params) {return '{d|'+params.data.value[1]+'}'+'\n'+'{c|'+params.data.value[0]+'}{b|'+params.data.name+'}';}"),
                    rich={
                        "b": {"fontSize": 18, "fontFamily": '宋体',"fontWeight":'normal',"padding":[0,0,-53,0]},
                        "c": {"fontSize": 18, "fontFamily": 'Bahnschrift SemiLight Condensed',"fontWeight":'normal'},
                        "r": {"fontSize": 18, "fontFamily": '宋体',"fontWeight":'normal',"padding":[0,0,-4,0]},
                        "d": {"fontSize": 30, "lineHeight": 10, "fontFamily": 'Bahnschrift SemiLight Condensed',"fontWeight":'normal'},
                    })
             ) 
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False))#去掉图例
    )
    return b

In [21]:
my_pie1([('sa',1),('s',2)]).render_notebook()

In [6]:
arr1=[1,2]
arr=arr1.copy()

In [238]:
y_value= [4,
148,
373,
414,
438,
511,
]
y_percent = [
    10.01,
    20.02,
    0.97,
    19,
    25,
    25
]
y = []
for i in range(len(y_value)):
    y.append({'value':y_value[i],'percent':y_percent[i]})

In [239]:
y

[{'value': 4, 'percent': 10.01},
 {'value': 148, 'percent': 20.02},
 {'value': 373, 'percent': 0.97},
 {'value': 414, 'percent': 19},
 {'value': 438, 'percent': 25},
 {'value': 511, 'percent': 25}]

In [130]:
save_doc('result/test.docx',True)

#### 3.生源地结构

#### 4.民族结构

In [20]:
data1=get_group_order_table(data,L('生源省份'),need_sum=False)

        sysf   num proportion
0        上海市     6      0.11%
1        云南省    45      0.84%
2     内蒙古自治区    55      1.03%
3        北京市     5      0.09%
4        吉林省    21      0.39%
5        四川省    54      1.01%
6        天津市    16      0.30%
7    宁夏回族自治区    47      0.88%
8        安徽省    88      1.64%
9        山东省   105      1.96%
10       山西省   184      3.43%
11       广东省    19      0.35%
12   广西壮族自治区    42      0.78%
13  新疆维吾尔自治区    84      1.57%
14       江苏省    68      1.27%
15       江西省    60      1.12%
16       河北省   115      2.14%
17       河南省   171      3.19%
18       浙江省    16      0.30%
19       海南省    33      0.62%
20       湖北省    29      0.54%
21       湖南省    37      0.69%
22       甘肃省   150      2.80%
23       福建省    20      0.37%
24     西藏自治区    10      0.19%
25       贵州省    65      1.21%
26       辽宁省    20      0.37%
27       重庆市    63      1.17%
28       陕西省  3683     68.65%
29       青海省    31      0.58%
30      黑龙江省    23      0.43%
        sysf   num proportion
3        北

In [29]:
data1[['sysf','num']].values.tolist()

[['北京市', 5],
 ['上海市', 6],
 ['西藏自治区', 10],
 ['天津市', 16],
 ['浙江省', 16],
 ['广东省', 19],
 ['辽宁省', 20],
 ['福建省', 20],
 ['吉林省', 21],
 ['黑龙江省', 23],
 ['湖北省', 29],
 ['青海省', 31],
 ['海南省', 33],
 ['湖南省', 37],
 ['广西壮族自治区', 42],
 ['云南省', 45],
 ['宁夏回族自治区', 47],
 ['四川省', 54],
 ['内蒙古自治区', 55],
 ['江西省', 60],
 ['重庆市', 63],
 ['贵州省', 65],
 ['江苏省', 68],
 ['新疆维吾尔自治区', 84],
 ['安徽省', 88],
 ['山东省', 105],
 ['河北省', 115],
 ['甘肃省', 150],
 ['河南省', 171],
 ['山西省', 184],
 ['陕西省', 3683]]

# 地图研究

In [70]:
make_snapshot(snapshot,picture_china_map(data_zip), f'result/1.png',pixel_ratio = 5)

In [95]:
picture_china_map(data_zip)

'E:\\桌面\\git_code\\myCode\\render.html'

In [35]:
data_zip=[['北京', 1], ['上海', 2], ['西藏', 3], ['天津', 4], ['浙江', 5], ['广东', 6], ['辽宁', 7], ['福建', 8], ['吉林', 9], ['黑龙江', 10], ['湖北', 11], ['青海', 12], ['海南', 13], ['湖南', 14], ['广西', 15], ['云南', 16], ['宁夏', 17], ['四川', 18], ['内蒙古', 19], ['江西', 20], ['重庆', 21], ['贵州', 22], ['江苏', 23], ['新疆', 24], ['安徽', 25], ['山东', 26], ['河北', 27], ['甘肃', 28], ['河南', 29], ['山西', 30], ['陕西', 31]]